<a href="https://colab.research.google.com/github/RENISA-GRACELIN/Online_Internship_Projects/blob/main/R_SVM_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import os
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import layers
from keras.applications.vgg16 import preprocess_input
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Data directories
train_dir = '/content/drive/MyDrive/DR_50/Train'
test_dir = '/content/drive/MyDrive/DR_50/Test'
valid_dir = '/content/drive/MyDrive/DR_50/Valid'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Image dimensions
img_width, img_height = 300, 300  # Adjust according to your image dimensions
batch_size = 32
num_epochs = 20

In [5]:
class SelfAttention(layers.Layer):
    def __init__(self, attention_size):
        super(SelfAttention, self).__init__()
        self.attention_size = attention_size

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], self.attention_size),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(self.attention_size,),
                                 initializer="zeros")
        self.u = self.add_weight(name="att_u", shape=(self.attention_size,),
                                 initializer="normal")
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        v = tf.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        vu = tf.tensordot(v, self.u, axes=1)
        weights = tf.nn.softmax(vu)
        output = tf.reduce_sum(x * tf.expand_dims(weights, -1), 1)

        return output


In [6]:
class SpatialAttention(layers.Layer):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = layers.Conv2D(64, kernel_size, padding='same', activation='relu')
        self.conv2 = layers.Conv2D(1, kernel_size, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_out = tf.reduce_mean(inputs, axis=3)
        max_out = tf.reduce_max(inputs, axis=3)
        x = tf.stack([avg_out, max_out], axis=3)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

In [7]:
# Load VGG16 model for feature extraction
model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
# Function to extract features from an image
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features

In [16]:
# Load images and extract features
x_train, y_train = [], []
for i in range(5):
    class_dir = os.path.join(train_dir, str(i))
    for img_path in os.listdir(class_dir):
        features = extract_features(os.path.join(class_dir, img_path))
        x_train.append(features)
        y_train.append(i)


1/1 [==============================] - 0s 469ms/step


In [17]:
from tqdm import tqdm

# Evaluate on test set
x_test, y_test = [], []
for i in range(5):
    class_dir = os.path.join(test_dir, str(i))
    # Use tqdm to create a progress bar
    for img_path in tqdm(os.listdir(class_dir), desc=f'Class {i}'):
        features = extract_features(os.path.join(class_dir, img_path))
        x_test.append(features)
        y_test.append(i)

#y_pred = clf.predict(x_test)


Class 0:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 494ms/step


Class 0:  20%|██        | 1/5 [00:00<00:02,  1.69it/s]

1/1 [==============================] - 0s 473ms/step


Class 0:  40%|████      | 2/5 [00:01<00:01,  1.74it/s]

...

Class 4:  60%|██████    | 3/5 [00:01<00:01,  1.75it/s]

1/1 [==============================] - 0s 479ms/step


Class 4:  80%|████████  | 4/5 [00:02<00:00,  1.75it/s]

1/1 [==============================] - 0s 471ms/step


Class 4: 100%|██████████| 5/5 [00:02<00:00,  1.77it/s]


In [30]:
# Flatten the features
x_train_flat = np.array([feature.flatten() for feature in x_train])
x_test_flat = np.array([feature.flatten() for feature in x_test])

# List of kernels to try
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernels:
    print(f"Training SVM with {kernel} kernel...")

    # Train SVM classifier
    clf = svm.SVC(kernel=kernel)
    clf.fit(x_train_flat, y_train)

    # Predict on test set
    y_pred = clf.predict(x_test_flat)


    # Predict on test set
    y_pred = clf.predict(x_test_flat)

    # Print performance metrics
    print(f'Performance metrics for {kernel} kernel:')
    print('Classification Report:')
    print(metrics.classification_report(y_test, y_pred))

    # Additional performance metrics
    accuracy = metrics.accuracy_score(y_test, y_pred)
    sensitivity = metrics.recall_score(y_test, y_pred, average='macro')
    precision = metrics.precision_score(y_test, y_pred, average='macro')
    f1 = metrics.f1_score(y_test, y_pred, average='macro')

    print(f'Accuracy: {accuracy}')
    print(f'Sensitivity: {sensitivity}')
    print(f'Precision: {precision}')
    print(f'F1 Score: {f1}\n')


Training SVM with linear kernel...
Performance metrics for linear kernel:
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.60      0.60         5
           1       1.00      0.60      0.75         5
           2       0.57      0.80      0.67         5
           3       0.25      0.20      0.22         5
           4       0.83      1.00      0.91         5

    accuracy                           0.64        25
   macro avg       0.65      0.64      0.63        25
weighted avg       0.65      0.64      0.63        25

Accuracy: 0.64
Sensitivity: 0.64
Precision: 0.6509523809523811
F1 Score: 0.6295959595959596

Training SVM with poly kernel...
Performance metrics for poly kernel:
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.80      0.47         5
           1       1.00      0.60      0.75         5
           2       0.50      0.40      0.44         5
           

In [19]:
# Print performance metrics
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.60      0.40         5
           1       1.00      0.60      0.75         5
           2       0.00      0.00      0.00         5
           3       0.20      0.20      0.20         5
           4       1.00      1.00      1.00         5

    accuracy                           0.48        25
   macro avg       0.50      0.48      0.47        25
weighted avg       0.50      0.48      0.47        25

Confusion Matrix:
[[3 0 0 2 0]
 [1 3 0 1 0]
 [4 0 0 1 0]
 [2 0 2 1 0]
 [0 0 0 0 5]]


In [29]:
 # Print performance metrics
print(f'Performance metrics for {kernel} kernel:')
print('Classification Report:')
print(metrics.classification_report(y_test, y_pred))

from sklearn.preprocessing import LabelBinarizer

# Convert y_test and y_pred to binary/multilabel indicator format
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)
y_pred_bin = lb.transform(y_pred)

# Now you can calculate the AUC
auc = metrics.roc_auc_score(y_test_bin, y_pred_bin,
                            multi_class='ovr')

# Additional performance metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')
sensitivity = metrics.recall_score(y_test, y_pred, average='macro')
precision = metrics.precision_score(y_test, y_pred, average='macro')
f1 = metrics.f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy}')
print(f'AUC: {auc}')
print(f'Sensitivity: {sensitivity}')
print(f'Precision: {precision}')
print(f'F1 Score: {f1}\n')

Performance metrics for sigmoid kernel:
Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.60      0.40         5
           1       1.00      0.60      0.75         5
           2       0.00      0.00      0.00         5
           3       0.20      0.20      0.20         5
           4       1.00      1.00      1.00         5

    accuracy                           0.48        25
   macro avg       0.50      0.48      0.47        25
weighted avg       0.50      0.48      0.47        25



AxisError: axis 1 is out of bounds for array of dimension 1